Punto 1

In [99]:
import pandas as pd

data = pd.read_csv('interseccion_zonas_buffer.csv')
df = data.copy()

# Convierte CODCOMP a string con 0 a la izquierda si departamento <= 9 
df['CODCOMP_STR'] = (df['CODCOMP'].astype(str)).str.zfill(10)

# Convierte CODLOC a string con 0 a la izquierda si departamento <= 9 
df['CODLOC_STR'] = (df['CODLOC'].astype(str)).str.zfill(5)

# Elimina las columnas fid, NOMLOC y CODCOMP_A
df.drop(columns=['fid', 'NOMLOC', 'CODCOMP_A', 'CODCOMP', 'CODLOC'], inplace=True)

# Se crea el dataframe df_farmacias_depto a partir de las columnas NOMBDEPTO, NOMBRE, DIRECCION
df_farmacias_depto = df[['NOMBDEPTO', 'NOMBRE', 'DIRECCION']]
# Crea la columna CANT_FARMACIAS con la concatenación de NOMBRE y DIRECCION de la farmacia
df_farmacias_depto['CANT_FARMACIAS'] = df_farmacias_depto['NOMBRE'] + df_farmacias_depto['DIRECCION']
df_farmacias_depto.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)
# Elimina filas duplicadas
df_farmacias_depto = df_farmacias_depto.drop_duplicates()
# Cuenta farmacias por nombre de departamento
df_farmacias_depto = df_farmacias_depto.groupby(by = ['NOMBDEPTO']).count()
df_farmacias_depto.to_csv('farmacias_departamento.csv')
print(df_farmacias_depto)

C:\Users\cchabay\AppData\Local\Temp\ipykernel_11496\3375835547.py:3: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('interseccion_zonas_buffer.csv')


                CANT_FARMACIAS
NOMBDEPTO                     
ARTIGAS                     12
CANELONES                   72
CERRO LARGO                  9
COLONIA                     36
DURAZNO                     11
FLORES                       9
FLORIDA                     15
LAVALLEJA                   13
MALDONADO                   48
MONTEVIDEO                 172
PAYSANDU                    15
RIO NEGRO                   16
RIVERA                      12
ROCHA                       19
SALTO                       12
SAN JOSE                    20
SORIANO                     21
TACUAREMBO                  23
TREINTA Y TRES               9


C:\Users\cchabay\AppData\Local\Temp\ipykernel_11496\3375835547.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_farmacias_depto['CANT_FARMACIAS'] = df_farmacias_depto['NOMBRE'] + df_farmacias_depto['DIRECCION']
C:\Users\cchabay\AppData\Local\Temp\ipykernel_11496\3375835547.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_farmacias_depto.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)


Punto 2

In [116]:
# Se crea el dataframe df_localidad_poblacion con poblacion x localidad
df_localidad_poblacion = df[['CODCOMP_STR', 'CODLOC_STR', 'NOMBLOC', 'POBLACION']]
df_localidad_poblacion = df_localidad_poblacion.drop_duplicates()
df_localidad_poblacion.drop(columns=['CODCOMP_STR'], inplace=True)
df_localidad_poblacion = df_localidad_poblacion.groupby(by = ['CODLOC_STR', 'NOMBLOC']).sum()
df_localidad_poblacion = df_localidad_poblacion.sort_values(by='CODLOC_STR')
df_localidad_poblacion = df_localidad_poblacion.reset_index()
#print(df_localidad_poblacion[])

# Se crea el dataframe df_localidad_farmacias a partir de las columnas CODLOC, NOMBRE, DIRECCION
df_localidad_farmacias = df[['CODLOC_STR', 'NOMBRE', 'DIRECCION']]
# Crea la columna CANT_FARMACIAS con la concatenación de NOMBRE y DIRECCION de la farmacia
df_localidad_farmacias['CANT_FARMACIAS'] = df_localidad_farmacias['NOMBRE'] + df_localidad_farmacias['DIRECCION']
# Elimina columnas innecesarias
df_localidad_farmacias.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)
# Elimina filas duplicadas
df_localidad_farmacias = df_localidad_farmacias.drop_duplicates()
# Cuenta farmacias por Localidad
df_localidad_farmacias = df_localidad_farmacias.groupby(by = ['CODLOC_STR']).count()
df_localidad_farmacias = df_localidad_farmacias.sort_values(by='CODLOC_STR')
df_localidad_farmacias = df_localidad_farmacias.reset_index()
#print(df_localidad_farmacias)

df_localidad_farmacias.drop(columns=['CODLOC_STR'], inplace=True)
df_loc_pob_farm = pd.concat([df_localidad_poblacion, df_localidad_farmacias], axis = 1)
print(df_loc_pob_farm.query('POBLACION > 3000 and CANT_FARMACIAS == 0'))
df_loc_pob_farm.query('POBLACION > 3000 and CANT_FARMACIAS == 0').to_csv('loc_grandes_sin_farm.csv')
print(df_loc_pob_farm.query('POBLACION < 3000 and CANT_FARMACIAS != 0'))
df_loc_pob_farm.query('POBLACION < 3000 and CANT_FARMACIAS != 0').to_csv('loc_chicas_con_farm.csv')

#print(df_localidad_poblacion)
#df_localidad_poblacion.set_index('CODLOC_STR').join(df_localidad_farmacias.set_index('CODLOC_STR'))
#df_loc_pob_farm = pd.merge(df_localidad_poblacion, df_localidad_farmacias, on='CODLOC_STR', how='inner')
#print(df_loc_pob_farm)
#print(df_localidad_farmacias['CANT_FARMACIAS'].sum())

df_loc_pob_farm.to_csv('loc_pob_farm.csv')

#df_localidad_poblacion = df_localidad_poblacion.groupby(by = ['CODLOC', 'NOMBLOC']).sum()

# Se crea el dataframe df_farmacias_localidad a partir de las columnas CODLOC, NOMB, POBLACION, NOMBRE, DIRECCION
#df_farmacias_loc = df[['CODCOMP', 'CODLOC','NOMBLOC', 'POBLACION', 'NOMBRE', 'DIRECCION']]
# Crea la columna NOMDIR con la concatenación de NOMBRE y DIRECCION de la farmacia
#df_farmacias_loc['NOMDIR'] = df_farmacias_loc['NOMBRE'] + df_farmacias_loc['DIRECCION']

#df_farmacias_loc.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)

# Elimina filas duplicadas
#df_farmacias_loc = df_farmacias_loc.drop_duplicates()

#df_farmacias_loc = df_farmacias_loc.groupby(by = ['CODCOMP', 'CODLOC', 'NOMBLOC']).sum()
#print(df_farmacias_loc)

# Cuenta farmacias por localidad
#df_farmacias_loc = df_farmacias_loc.groupby(by = ['CODLOC', 'NOMBLOC', 'POBLACION']).count()
#df_farmacias_loc.to_csv('farmacias_localidad.csv')
#print(df_farmacias_loc)

    CODLOC_STR                     NOMBLOC  POBLACION  CANT_FARMACIAS
46       03524              JOAQUIN SUAREZ     6570.0               0
52       03530  FRACC. CNO. ANDALUZ Y R.84     9295.0               0
71       03726                  LAS TOSCAS     3146.0               0
77       03733            VILLA AEROPARQUE     4307.0               0
124      03959                    NEPTUNIA     4774.0               0
150      04521               FRAILE MUERTO     3168.0               0
314      10821                CERRO PELADO     8177.0               0
363      11522              NUEVO PAYSANDU     8578.0               0
393      11951         CHACRAS DE PAYSANDU     3965.0               0
433      13836                  LA PEDRERA     3363.0               0
434      13837                     MANDUBI     6019.0               0
    CODLOC_STR                        NOMBLOC  POBLACION  CANT_FARMACIAS
5        02522                TOMAS GOMENSORO     2659.0               2
6        02621

C:\Users\cchabay\AppData\Local\Temp\ipykernel_11496\1880104137.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_localidad_farmacias['CANT_FARMACIAS'] = df_localidad_farmacias['NOMBRE'] + df_localidad_farmacias['DIRECCION']
C:\Users\cchabay\AppData\Local\Temp\ipykernel_11496\1880104137.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_localidad_farmacias.drop(columns=['NOMBRE', 'DIRECCION'], inplace=True)


Punto 3

In [ ]:
# Se crea el dataframe df_poblacion_farmacia a partir de las columnas NOMBRE, DIRECCION, x, y, POBLACION
df_poblacion_farmacia = df[['NOMBRE', 'DIRECCION', 'x', 'y', 'POBLACION']]
df_poblacion_farmacia = df_poblacion_farmacia.groupby(by = ['NOMBRE', 'DIRECCION', 'x', 'y']).sum()
df_poblacion_farmacia.to_csv('poblacion_farmacia.csv')
print(df_poblacion_farmacia)
